In [1]:
#  Imports
import pandas as pd
import os

# 경로 설정
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..')) 
data_dir = os.path.join(base_dir, 'data')  # data 폴더 경로
processed_dir = os.path.join(data_dir, 'processed')  # processed 폴더 경로

# 파일 경로 설정
climate_data_path = os.path.join(processed_dir, 'preprocessed_climate.csv')
market_data_path = os.path.join(processed_dir, 'preprocessed_agromarket.csv')

# 데이터 불러오기
dfc = pd.read_csv(climate_data_path)
dfm = pd.read_csv(market_data_path)

In [2]:
dfc.head()

,지점명,일시,평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),평균 현지기압(hPa),합계 일조시간(hr),일 최심신적설(cm)
0,춘천,2019-01-03,-7.1,-13.7,2.3,NaN,1.6,0.7,59.4,1025.8,8.0,NaN
1,춘천,2019-01-04,-4.6,-12.0,2.9,NaN,3.1,0.9,65.1,1020.9,6.6,NaN
2,춘천,2019-01-05,-4.3,-9.5,3.4,NaN,2.5,0.9,59.0,1017.1,7.9,NaN
3,춘천,2019-01-06,-5.1,-10.4,4.0,NaN,2.3,0.9,54.0,1019.0,6.6,NaN
4,춘천,2019-01-07,-5.1,-12.1,3.3,NaN,4.4,1.1,57.6,1018.5,6.8,NaN


In [3]:
#  날짜 컬럼 변환
dfc['일시'] = pd.to_datetime(dfc['일시'], format='%Y-%m-%d')
dfm['일자'] = pd.to_datetime(dfm['일자'].astype(str), format='%Y%m%d')

# 2024-12-20 이후의 데이터 필터링
dfm = dfm[dfm['일자'] < '2024-12-20']
dfc = dfc[dfc['일시'] < '2024-12-20']

In [4]:
#  서울/수도권 도매시장 필터링
# 서울 및 수도권 도매시장 필터링
capital_region_markets = ['구리', '서울가락', '서울강서', '수원', '인천남촌', '인천삼산', '안산', '안양']
dfm = dfm[dfm['도매시장'].isin(capital_region_markets)]

# 서울 지역 날씨 필터링
dfc = dfc[dfc['지점명'] == '서울']

In [5]:
# 품목별 단위가격 계산 및 일자별 집계
dfm['단위가격(원/kg)'] = dfm['금액(원)'] / dfm['물량(kg)']

# 품목별 일자별 총물량과 평균단가 계산
dfm_daily = (
    dfm.groupby(['품목', '일자'])
    .agg(총물량=('물량(kg)', 'sum'), 평균단가=('단위가격(원/kg)', 'mean'))
    .reset_index()
)


In [6]:
# 기후 파생변수 생성
# 온도차 파생변수
dfc['온도차(°C)'] = (dfc['최고기온(°C)'] - dfc['최저기온(°C)']).round(1) 

# 계절 파생변수
def get_season(month):
    if month in [3, 4, 5]:
        return '봄'
    elif month in [6, 7, 8]:
        return '여름'
    elif month in [9, 10, 11]:
        return '가을'
    else:
        return '겨울'

dfc['계절'] = pd.to_datetime(dfc['일시']).dt.month.apply(get_season)


In [7]:
# 필요한 칼럼만 남기기
dfc = dfc[['일시', '온도차(°C)', '평균기온(°C)', '일강수량(mm)', '계절']]

In [8]:
# 결측치 처리
dfc.fillna(0, inplace=True)

In [9]:
# 데이터 결합
dfc = dfc.rename(columns={'일시': '일자'})
df_final = pd.merge(dfm_daily, dfc, on='일자', how='left')

In [10]:
# 데이터 저장
final_data_path = os.path.join(processed_dir, 'daily_average_price_volume_by_product_seoul.csv')
df_final.to_csv(final_data_path, index=False, encoding='utf-8-sig')

In [11]:
#  기본 정보 확인
print("=== 최종 df_final 기본 정보 ===")
print(df_final.info())
print(df_final.head(10))

=== 최종 df_final 기본 정보 ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53244 entries, 0 to 53243
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   품목        53244 non-null  object        
 1   일자        53244 non-null  datetime64[ns]
 2   총물량       53244 non-null  float64       
 3   평균단가      53244 non-null  float64       
 4   온도차(°C)   53244 non-null  float64       
 5   평균기온(°C)  53244 non-null  float64       
 6   일강수량(mm)  53244 non-null  float64       
 7   계절        53244 non-null  object        
dtypes: datetime64[ns](1), float64(5), object(2)
memory usage: 3.2+ MB
None
   품목         일자        총물량         평균단가  온도차(°C)  평균기온(°C)  일강수량(mm)  계절
0  감귤 2019-01-03  1797298.1  3241.550629     11.6      -3.5       0.0  겨울
1  감귤 2019-01-04  1356605.5  3241.630229     10.3      -1.1       0.0  겨울
2  감귤 2019-01-05  1306249.6  3246.549462      6.6      -2.8       0.0  겨울
3  감귤 2019-01-07  1519848.7  3280.